# Move scans into organized patient folders

In [1]:
from importlib import reload
import pdf_ocr
reload(pdf_ocr)
from pdf_ocr import *

unsorted_scans = '/Users/byukan/docsort/data/_unsorted_scans'
file = os.listdir(unsorted_scans)[0]
p = f'{unsorted_scans}/{file}'

num_pages, ocr_text = pdf_ocr(p, reprocess=True)
names_counter = extract_name(ocr_text)

Saving 2 png images for file: 202012171708.pdf.


Running OCR on images and saving 2 .txt files for file: 202012171708.pdf.

In [2]:
file

'202012171708.pdf'

In [1]:
# names_counter

In [4]:
patients = pd.read_csv('data/patient_names.csv')
patient_names_set = set(patients.firstname + ' ' + patients.lastname)
doctors = pd.read_csv('data/doctor_names.csv')
doctor_names_set = set(doctors.firstname + ' ' + doctors.lastname)

In [5]:
patient_names_set.add('Brant Yukan')

In [6]:
def best_guess(names_counter, patient_names_set, doctor_names_set):
    """
    Favor patient names from known list and omit doctors names.
    :param: names_counter is the counter of PERSON entities.
    :return: patient's first and last name
    """
    # set(candidates).intersection(patient_names_set)
    candidates = list(set(x[0] for x in names_counter) - doctor_names_set) # remove doctor names
    fuzz_scores = [*map(lambda x: process.extractOne(x, patient_names_set, ), candidates)]

    first = sorted([x for x in zip(candidates, fuzz_scores)], key=lambda x: x[1][1], reverse=True)[0]
    score = first[1][1]
    if score > 95:
        return first[0]
    else:
        return 'No match found.'


name = best_guess(names_counter, patient_names_set, doctor_names_set)

In [2]:
first_name = ' '.join(name.split(' ')[:-1])
last_name = name.split(' ')[-1]

date = parse(file.split('.')[0]).date()
slash_date = date.strftime('%m/%d/%Y')

DOB = 'DOB'

folder_name = f'{last_name}, {first_name} - {DOB}'
folder_name

In [8]:
dot_date = date.strftime('%m.%d.%Y')
classification = 'Notice of Depo'
file_name = f'{dot_date} - {last_name}, {first_name} - {classification}.pdf'

Folder naming schema: 
Lastname, Firstname - MM_DD_YYYY

File naming schema:
DATE_OF_DOCUMENT - Lastname, Firstname - CLASSIFICATION.extracts
**Date_OF_DOCUMENT & CLASSIFICATION ARE FUTURE implementations.

In [3]:
# folder_name

In [4]:
import shutil

folder_path = f'data/patients/{folder_name}'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
shutil.copy(p, folder_path)

In [11]:
os.rename(f'{folder_path}/{file}', f'{folder_path}/{file_name}')

In [20]:
# creds = yaml.load(open('ssl/credentials.yml'), Loader=yaml.FullLoader)

# # https://github.com/pahaz/sshtunnel#example-3

# import mysql.connector
# import sshtunnel
# import pandas as pd

# sshtunnel.SSH_TIMEOUT = 5.0
# sshtunnel.TUNNEL_TIMEOUT = 5.0

# with sshtunnel.SSHTunnelForwarder(
#     ('zmbies.com',187),
#     ssh_username='root', ssh_password=creds['ssh_password'],
#     remote_bind_address=('127.0.0.1',3306)
#     ) as tunnel:
#         connection = mysql.connector.connect(user='root', password=creds['password'],host='127.0.0.1', port=tunnel.local_bind_port,database='myuser$mydb')
#         sql = 'SELECT * from patients'
#         df = pd.read_sql(sql, con=connection)
#         print(df.head())
#         connection.close()

In [21]:
# import mysql.connector
# import sshtunnel

# with sshtunnel.SSHTunnelForwarder(
#     (creds['host'], creds['port']),
#     ssh_username = creds['user'],
#     ssh_password = creds['ssh_password'],
#     remote_bind_address = ('127.0.0.1', 3306)
# ) as tunnel:
#     connection = mysql.connector.connect(
#         user = creds['user'],
#         password = creds['password'],
#         host = 'localhost',
#         port = tunnel.local_bind_port,
#         database = creds['database'],
#     )
#     mycursor = connection.cursor()
#     query = "SELECT * FROM patients"
#     mycursor.execute(query)
